## Setup

*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

Install requirements:


In [ ]:
!pip install --upgrade 'ocdskingfishercolab<0.4' psycopg2-binary > pip.log

Import packages and load extensions:

In [ ]:
import getpass
import matplotlib.pyplot as plt
import pandas as pd
from google.colab.data_table import DataTable
from ocdskingfishercolab import (
    format_thousands,
    list_collections,
    list_source_ids,
    render_json,
    save_dataframe_to_sheet,
    save_dataframe_to_spreadsheet,
    set_dark_mode,
    set_light_mode,
    set_search_path,
    set_spreadsheet_name,
)

# Load https://pypi.org/project/ipython-sql/
%load_ext sql
# Load https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table

Set the name of the spreadsheet to export results to:

In [ ]:
spreadsheet_name = 'feedback_results'

set_spreadsheet_name(spreadsheet_name)

Enter your PostgreSQL credentials and connect to the Kingfisher Process database:

In [ ]:
user = input('Username:')
password = getpass.getpass('Password:')

# Don't show connection string after execute.
%config SqlMagic.displaycon = False

connection_string = 'postgresql://' + user + ':' + password + '@postgres.kingfisher.open-contracting.org/kingfisher_process?sslmode=require'
%sql $connection_string

Configure the notebook environment:

In [ ]:
# Increase max columns so that Pandas DataFrames with many columns are rendered as data tables.
DataTable.max_columns = 50
# Remove the index from data tables for easier copy-pasting to Google Docs.
DataTable.include_index = False

# Return Pandas DataFrames instead of regular result sets.
%config SqlMagic.autopandas = True
# Don't print number of rows affected.
%config SqlMagic.feedback = False

# If you set Tools > Settings > Site > Theme to dark, uncomment this line.
# set_dark_mode()
# If you are creating plots to copy-paste into reports, uncomment this line.
# set_light_mode()